In [48]:
import pandas as pd
import numpy as np

In [2]:
game_stats=pd.read_csv('game_stats_results.csv')
nlp=pd.read_csv('nlp_results.csv')

In [3]:
columns=['home_team','away_team','final_result','predict_stats','proba_stats_home','proba_stats_draw','proba_stats_away']
game_stats=game_stats.loc[:,columns]

In [4]:
columns=['team_home','team_away','result','predict_comments','proba_home','proba_draw','proba_away']
nlp=nlp.loc[:,columns]

In [52]:
def get_stats(row,column):
    home = row['team_home']
    away = row['team_away']
    df=game_stats.query(f'(home_team == "{home}") & (away_team == "{away}")')
    if len(df)==0:
        return np.nan
    else:
        return df[column].values[0]

In [57]:
df=nlp
columns=['predict_stats','proba_stats_home', 'proba_stats_draw', 'proba_stats_away']
for column in columns:
    df[column]=df.apply(lambda row: get_stats(row,column), axis=1)
df=df.dropna()
df

,team_home,team_away,result,predict_comments,proba_home,proba_draw,proba_away,predict_stats,proba_stats_home,proba_stats_draw,proba_stats_away
0,B. Monchengladbach,Schalke,Home,1,0.366510,0.337604,0.295886,0.0,0.232903,0.429677,0.337419
1,RB Leipzig,Schalke,Away,-1,0.312016,0.123761,0.564223,-1.0,0.040970,0.120401,0.838629
3,Freiburg,RB Leipzig,Draw,1,0.435557,0.234628,0.329815,1.0,0.541892,0.179054,0.279054
4,Dortmund,Schalke,Home,0,0.305514,0.404616,0.289870,1.0,0.716892,0.154054,0.129054
5,FC Koln,Paderborn,Home,1,0.486160,0.183059,0.330782,1.0,0.525000,0.234375,0.240625
...,...,...,...,...,...,...,...,...,...,...,...
226,Bayer Leverkusen,RB Leipzig,Draw,1,0.408346,0.205717,0.385937,1.0,0.362903,0.309677,0.327419
227,Mainz,Schalke,Draw,0,0.269425,0.375181,0.355394,0.0,0.232903,0.429677,0.337419
228,FC Koln,Hoffenheim,Away,1,0.437861,0.299240,0.262898,1.0,0.541892,0.179054,0.279054
229,RB Leipzig,Wolfsburg,Draw,-1,0.339227,0.152672,0.508101,0.0,0.232903,0.429677,0.337419
